In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Mexico_Covid.csv', sep=';')

In [3]:
df = df.iloc[:, 1:]

In [4]:
df

,pneumonia,age,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,covid_res,Deaths
0,0,27,0,0,0,0,0,0,0,0,0,0,2,0
1,0,24,0,0,0,0,0,0,0,0,0,0,2,0
2,0,54,0,0,0,0,0,0,0,2,0,0,2,0
3,2,30,0,0,0,0,0,0,0,0,0,0,2,0
4,0,60,2,0,0,0,2,0,2,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566597,0,58,2,0,0,0,0,0,0,0,0,0,0,0
566598,0,48,0,0,0,0,0,0,0,0,0,0,0,0
566599,2,49,0,0,0,0,0,0,0,2,0,0,0,0
566600,2,43,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566602 entries, 0 to 566601
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   pneumonia       566602 non-null  int64
 1   age             566602 non-null  int64
 2   diabetes        566602 non-null  int64
 3   copd            566602 non-null  int64
 4   asthma          566602 non-null  int64
 5   inmsupr         566602 non-null  int64
 6   hypertension    566602 non-null  int64
 7   other_disease   566602 non-null  int64
 8   cardiovascular  566602 non-null  int64
 9   obesity         566602 non-null  int64
 10  renal_chronic   566602 non-null  int64
 11  tobacco         566602 non-null  int64
 12  covid_res       566602 non-null  int64
 13  Deaths          566602 non-null  int64
dtypes: int64(14)
memory usage: 60.5 MB


In [6]:
df.isna().sum()

pneumonia         0
age               0
diabetes          0
copd              0
asthma            0
inmsupr           0
hypertension      0
other_disease     0
cardiovascular    0
obesity           0
renal_chronic     0
tobacco           0
covid_res         0
Deaths            0
dtype: int64

In [7]:
df.Deaths.value_counts(normalize=True)

0    0.936153
1    0.063847
Name: Deaths, dtype: float64

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
model = LogisticRegression()

In [10]:
from sklearn.model_selection import cross_val_score

In [11]:
X = df.drop('Deaths', axis = 1).values

In [12]:
y = df.Deaths.values

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [15]:
model.fit(X_train, y_train) 

C:\Users\toluh\anaconda3\envs\covenmlcohort\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
model.score(X_test, y_test)

0.9376726290802234

In [17]:
PredY = model.predict(X_test)

In [18]:
PredY_series = pd.Series(PredY)
y_test_series = pd.Series(y_test)

In [19]:
confusion_matrix = pd.crosstab(y_test_series, PredY_series, rownames=['Actual'], 
                               colnames=['Predicted'])

In [20]:
confusion_matrix

Predicted,0,1
Actual,,
0,104574,1450
1,5613,1684


In [21]:
from sklearn.metrics import confusion_matrix, classification_report

In [22]:
print(classification_report(y_test, PredY))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97    106024
           1       0.54      0.23      0.32      7297

    accuracy                           0.94    113321
   macro avg       0.74      0.61      0.65    113321
weighted avg       0.92      0.94      0.93    113321



### Balance the data

In [23]:
#uncomment and run the code line below to install imblearn
#!pip install imblearn

     ------------------------------------ 257.7/257.7 kB 452.3 kB/s eta 0:00:00


In [24]:
from imblearn.over_sampling import RandomOverSampler

In [25]:
res = RandomOverSampler()

In [26]:
X_res, y_res = res.fit_resample(X, y)

In [27]:
df.Deaths.value_counts()

0    530426
1     36176
Name: Deaths, dtype: int64

In [28]:
from collections import Counter

In [29]:
Counter(y_res)

Counter({0: 530426, 1: 530426})

In [30]:
from imblearn.under_sampling import RandomUnderSampler

In [31]:
rus = RandomUnderSampler(random_state=42)

In [32]:
X_rus, y_rus = rus.fit_resample(X, y)

In [33]:
Counter(y_rus)

Counter({0: 36176, 1: 36176})